# Python based CCloud Producer Demo¶

How can I get some Wikipedia pageview data for timeseries analysis using OpenTSx into my Kafka Cluster?
https://wikitech.wikimedia.org/wiki/Analytics/AQS/Pageviews


In [6]:
import sys

In [12]:
!conda update --all -y 

/bin/sh: 1: conda: not found


In [13]:
!conda install -n python3 -y -c conda-forge spacy

/bin/sh: 1: conda: not found


In [9]:
!pip install --no-binary :all: "confluent-kafka[avro,json,protobuf]"

/bin/sh: 1: pip: not found


In [5]:
!pip install --no-binary :all: argparse
!pip install --no-binary :all: requests
!pip install --no-binary :all: certifi
!pip install mwviews
!pip install git+https://github.com/Commonists/pageview-api.git

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached argparse-1.4.0.tar.gz (70 kB)
  Preparing metadata (setup.py) ... done
Skipping wheel build for argparse, due to binaries being disabled for it.
    Running setup.py install for argparse ... done
/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.u

In [3]:
import argparse

from confluent_kafka import avro, KafkaError
from confluent_kafka.admin import AdminClient, NewTopic
from uuid import uuid4

ModuleNotFoundError: No module named 'confluent_kafka'

In [ ]:
class Wikipage(object):
    """
        Wikipage stores the meta-data for a Wikipage used in the analysis Kafka key.
    """

    # Use __slots__ to explicitly declare all data members.
    __slots__ = ["pagename", "id"]

    def __init__(self, pagename=None):
        self.pagename = pagename
        # Unique id used to track produce request success/failures.
        # Do *not* include in the serialized object.
        self.id = uuid4()

    @staticmethod
    def dict_to_name(obj, ctx):
        return Wikipage(obj['pagename'])

    @staticmethod
    def name_to_dict(name, ctx):
        return Wikipage.to_dict(name)

    def to_dict(self):
        """
            We must provide a dict representation of our class for serialization.
        """
        return dict(pagename=self.pagename)

In [ ]:
class Count(object):
    """
        Count stores the pageview counts.
    """

    # Use __slots__ to explicitly declare all data members.
    __slots__ = ["count", "id"]

    def __init__(self, count=None):
        self.count = count
        # Unique id used to track produce request success/failures.
        # Do *not* include in the serialized object.
        self.id = uuid4()

    @staticmethod
    def dict_to_count(obj, ctx):
        return Count(obj['count'])

    @staticmethod
    def count_to_dict(count, ctx):
        return Count.to_dict(count)

    def to_dict(self):
        """
            We must provide a dict representation of our class for serialization.
        """
        return dict(count=self.count)

In [ ]:
#
# Helper function to read CCloud configuration.
#
def read_ccloud_config(config_file):
    """Read Confluent Cloud configuration for librdkafka clients"""

    conf = {}
    with open(config_file) as fh:
        for line in fh:
            line = line.strip()
            if len(line) != 0 and line[0] != "#":
                parameter, value = line.strip().split('=', 1)
                conf[parameter] = value.strip()

    return conf

#
# Helper function to create Kafka Topic.
#
def create_topic(conf, topic):
    """
        Create a topic if needed
        Examples of additional admin API functionality:
        https://github.com/confluentinc/confluent-kafka-python/blob/master/examples/adminapi.py
    """

    a = AdminClient({
           'bootstrap.servers': conf['bootstrap.servers'],
           'sasl.mechanisms': 'PLAIN',
           'security.protocol': 'SASL_SSL',
           'sasl.username': conf['sasl.username'],
           'sasl.password': conf['sasl.password']
    })
    fs = a.create_topics([NewTopic(
         topic,
         num_partitions=1,
         replication_factor=3
    )])
    for topic, f in fs.items():
        try:
            f.result()  # The result itself is None
            print("Topic {} created".format(topic))
        except Exception as e:
            # Continue if error code TOPIC_ALREADY_EXISTS, which may be true
            # Otherwise fail fast
            if e.args[0].code() != KafkaError.TOPIC_ALREADY_EXISTS:
                print("Failed to create topic {}: {}".format(topic, e))
                sys.exit(1)

In [ ]:

# =============================================================================
#
# Produce messages to Confluent Cloud
#
# Using Confluent Python Client for Apache Kafka
# Writes JSON data, no integration with Confluent Cloud Schema Registry
#
# =============================================================================
from confluent_kafka import Producer
from confluent_kafka.serialization import StringSerializer

import json
import ccloud_lib

In [ ]:



# Define arguments and configurations and initialize

config_file = "/Users/mkaempf/.confluent/python.config"
topic = "test1"

conf = read_ccloud_config(config_file)
create_topic( conf, topic )

print( conf )

#
# The producer doesn't like some properties:
#
# for full list of configurations, see:
#  https://docs.confluent.io/platform/current/clients/confluent-kafka-python/#serializingproducer
conf.pop( 'schema.registry.url' )
conf.pop( 'basic.auth.credentials.source' )
conf.pop( 'basic.auth.user.info' )


In [ ]:
producer = Producer(conf)

delivered_records = 0

In [ ]:
# Optional per-message on_delivery handler (triggered by poll() or flush())
# when a message has been successfully delivered or
# permanently failed delivery (after retries).
def acked(err, msg):
    global delivered_records
    """Delivery report handler called on
    successful or failed delivery of message
    """
    if err is not None:
        print("Failed to deliver message: {}".format(err))
    else:
        delivered_records += 1
        print("Produced record to topic {} partition [{}] @ offset {}"
              .format(msg.topic(), msg.partition(), msg.offset()))
        

In [ ]:
pages = ["Paris", "London", "Berlin", "New-York" ]

import pageviewapi

from datetime import datetime

# datetime object containing current date and time
tStart = datetime.now()
 
print("now =", tStart)

# dd/mm/YY H:M:S
dt_string = tStart.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)

n = 1
for p in pages:
    n = n+1
    name_object = Name()
    name_object.name = p
    count_object = Count()
    count_object.count = n
    
    print("Producing JSON record: {}\t{}".format(name_object.name, count_object.count))
    
    data = pageviewapi.per_article('en.wikipedia', p, '20200101', '20201231',
                        access='all-access', agent='all-agents', granularity='daily')
    
    producer.produce(topic="topic1", key=json.dumps(name_object.to_dict()), value=json.dumps(data), on_delivery=acked)

    producer.poll(0)

producer.flush()

print("{} messages were produced to topic {}!".format(delivered_records, topic))

tEnd = datetime.now()
 
print("now =", tEnd)

print("loadtime => ", tEnd - tStart)

In [ ]:

pageviewapi.per_article('en.wikipedia', 'Paris', '20151106', '20151120',
                        access='all-access', agent='all-agents', granularity='daily')

In [ ]:
pageviewapi.legacy_pagecounts('fr.wikipedia', '2010010100', '2011010100', granularity='daily')